In [27]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json

### Side-Effects of Osimetrinib

#### Common
- Diarrhoea
- Eyelid Pruritus
- Increased risk of infection
- Nail Discolouration
- Nail Disorders
- Skin reactions
- Stomatitis

#### Uncommon
- Cutaneous Vasculitis
- Eye Disorders
- Eye Inflammation
- QT Interval Prolongation

#### Rare
- Stevens-Jhonson syndrome

#### Unknown Freq
- Decreased leucocytes
- Neutropenia
- Thrombocytopenia

#### Broad Text Searches for Side Effects
- Diarrhea
- Eye
- Skin, Cutaneous
- Nail 
- Infection
- Stomatitis
- QT 
- Stevens-Jhonson
- Neutropenia, Leukopenia
- Thrombocytopenia

In [28]:
sideeff_list = [['Diarrhea'],['Eye'],['Skin', 'Cutaneous'],['Nail'],['Infection'],['Stomatitis'],['QT'],['Stevens-Jhonson'],['Neutropenia', 'Leukopenia'],['Thrombocytopenia']]

In [29]:
icd10_list = pd.read_csv(f'{os.getcwd()}/Section111ValidICD10-Jan2023-DupFixed.csv')

In [30]:
icd10_list = icd10_list.rename(columns={
    'CODE' : 'code',
    'SHORT DESCRIPTION (VALID ICD-10 FY2023)' : 's_desc',
    'LONG DESCRIPTION (VALID ICD-10 FY2023)' : 'l_desc',
    'NF EXCL' : 'nf_excl'
})

In [31]:
icd10_list.loc[icd10_list.l_desc.isnull(), 'l_desc']

3495    NaN
3921    NaN
Name: l_desc, dtype: object

In [32]:
icd10_list.loc[icd10_list.l_desc.isnull(), 'l_desc'] = ['','']
icd10_list.loc[icd10_list.l_desc.isnull(), 'l_desc']

Series([], Name: l_desc, dtype: object)

In [33]:
substring = 'thrombocytopenia'
req_code_list = list(icd10_list[icd10_list.s_desc.str.contains(substring, case=False)].code)+\
                list(icd10_list[icd10_list.l_desc.str.contains(substring, case=False)].code)
req_code_list = list(set(req_code_list))

In [34]:
def find_diag_codes(substring:str):
    req_code_list = list(icd10_list[icd10_list.s_desc.str.contains(substring, case=False)].code)+\
                    list(icd10_list[icd10_list.l_desc.str.contains(substring, case=False)].code)
    return pd.Series(req_code_list, name='Codes').sort_values().reset_index(drop=True).to_list()

In [35]:
sideeff_codes = {}
for sideeff_strings in sideeff_list:
    _lst = []
    for sideeff in sideeff_strings:
        _lst = _lst + [code for code in find_diag_codes(sideeff)]
    sideeff_codes[sideeff_strings[0]] = _lst

/var/folders/gy/3csfmkt55nj5gqw1rm4_s1k00000gn/T/ipykernel_77176/2102634365.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(req_code_list, name='Codes').sort_values().reset_index(drop=True).to_list()


In [36]:
for key in sideeff_codes.keys():
    sideeff_codes[key] = list(set(sideeff_codes[key]))

In [37]:
with open(f'{os.getcwd()}/side_effect_codes.json', 'w+') as f:
    json.dump(sideeff_codes, f, indent=4, sort_keys=True)